# Setup Structrur Project

#### *SETUP Config*

In [1]:
import os

def setup():
    # Folder structure sesuai struktur yang diinginkan
    folders = [
        "configs",
        "data/raw",
        "data/processed",
        "data/datasets/cifar10",
        "data/datasets/mnist",
        "data/datasets/custom",
        "models/base_models",
        "models/trained_models",
        "models/injected_models",
        "models/detector",
        "src",
        "src/utils",
        "notebooks",
        "tests"
    ]
    files = {
        "configs/base_config.yaml": "",
        "configs/training_config.yaml": "",
        "configs/injection_config.yaml": "",
        "src/__init__.py": "",
        "src/data_acquisition.py": "",
        "src/model_training.py": "",
        "src/injection_engine.py": "",
        "src/feature_extractor.py": "",
        "src/detector_trainer.py": "",
        "src/utils/__init__.py": "",
        "src/utils/helpers.py": "",
        "src/utils/visualization.py": "",
        "notebooks/01_data_exploration.ipynb": "",
        "notebooks/02_injection_analysis.ipynb": "",
        "notebooks/03_detector_evaluation.ipynb": "",
        "tests/test_injection.py": "",
        "tests/test_features.py": "",
        "main.py": "",
        "requirements.txt": "",
        "README.md": ""
    }

    # Create folders
    for folder in folders:
        os.makedirs(folder, exist_ok=True)

    # Create files
    for file_path, content in files.items():
        dir_name = os.path.dirname(file_path)
        if dir_name and not os.path.exists(dir_name):
            os.makedirs(dir_name, exist_ok=True)
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(content)

#### # Jalankan fungsi untuk membuat struktur

In [2]:
setup()

# Ceck and Test GPU

In [15]:
import yaml
config_path="configs/base_config.yaml"
config=[]
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)
scratch_datasets = config['datasets']
scratch_models = config['base_models']
base_pretrained_models = []
for i, dataset in scratch_datasets.items():
    for model_name, i in scratch_models.items():
        base_pretrained_models.append(f"models/base_models/{model_name}_pretrained.pth")
# print(config['base_models'].items())
print(base_pretrained_models)

['models/base_models/resnet18_pretrained.pth']


In [37]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.1904, 0.9045, 0.6009],
        [0.9846, 0.4433, 0.5178],
        [0.8635, 0.2702, 0.4201],
        [0.6366, 0.5579, 0.2481],
        [0.4006, 0.8064, 0.0949]])


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
b, m, n, k = 1, 1024, 1024, 512
A = torch.randn(b, m, k, dtype=torch.float32, device=device)
print(A)


tensor([[[-0.2318, -0.8590, -0.7416,  ...,  0.1813, -0.0781,  0.4358],
         [ 0.8333,  0.7809, -0.3760,  ..., -0.0396,  0.9033, -0.6771],
         [-0.1775,  0.4905, -0.1696,  ..., -0.6316,  1.5319,  1.1901],
         ...,
         [-1.1592, -0.9153,  1.1092,  ..., -0.2088, -0.5704, -0.7982],
         [-0.5697,  1.5137, -1.8539,  ...,  1.0510,  0.6981, -0.0287],
         [-0.8787, -1.4977,  0.6994,  ...,  1.0186,  1.0458,  0.9592]]],
       device='cuda:0')


In [2]:
import torch
print(torch.__version__)              # Harus versi CUDA
print(torch.version.cuda)             # Harus '12.1'
print(torch.cuda.is_available())      # Harus True
# print(torch.cuda.get_device_name(0))  # Harus 'NVIDIA GeForce RTX 4060'

2.8.0+cu129
12.9
True


In [ ]:
import torch
print(torch.cuda.is_available())  # hards True
print(torch.cuda.get_device_name(0))  # Nama GPU

True
NVIDIA GeForce RTX 4060 Laptop GPU


In [40]:
import torch
import cudnn

# Prepare sample input data. nvmath-python accepts input tensors from pytorch, cupy, and
# numpy.
b, m, n, k = 1, 1024, 1024, 512
A = torch.randn(b, m, k, dtype=torch.float32, device="cuda")
B = torch.randn(b, k, n, dtype=torch.float32, device="cuda")
bias = torch.randn(b, m, 1, dtype=torch.float32, device="cuda")

result = torch.empty(b, m, n, dtype=torch.float32, device="cuda")

# Use the stateful Graph object in order to perform multiple matrix multiplications
# without replanning. The cudnn API allows us to fine-tune our operations by, for
# example, selecting a mixed-precision compute type.
graph = cudnn.pygraph(
   intermediate_data_type=cudnn.data_type.FLOAT,
   compute_data_type=cudnn.data_type.FLOAT,
)

a_cudnn_tensor    = graph.tensor_like(A)
b_cudnn_tensor    = graph.tensor_like(B)
bias_cudnn_tensor = graph.tensor_like(bias)

c_cudnn_tensor = graph.matmul(name="matmul", A=a_cudnn_tensor, B=b_cudnn_tensor)
d_cudnn_tensor = graph.bias(name="bias", input=c_cudnn_tensor, bias=bias_cudnn_tensor)

# Build the matrix multiplication. Building returns a sequence of algorithms that can be
# configured. Each algorithm is a JIT generated function that can be executed on the GPU.

graph.build([cudnn.heur_mode.A])
workspace = torch.empty(graph.get_workspace_size(), device="cuda", dtype=torch.uint8)

# Execute the matrix multiplication.
graph.execute(
   {
       a_cudnn_tensor: A,
       b_cudnn_tensor: B,
       bias_cudnn_tensor: bias,
       d_cudnn_tensor: result,
   },
   workspace
)

In [14]:
print(graph)

{
    "context": {
        "compute_data_type": "FLOAT",
        "intermediate_data_type": "FLOAT",
        "io_data_type": null,
        "name": "",
        "sm_count": -1
    },
    "cudnn_backend_version": "9.10.2",
    "cudnn_frontend_version": 11401,
    "json_version": "1.0",
    "nodes": [
        {
            "compute_data_type": "FLOAT",
            "inputs": {
                "A": "matmul::A",
                "B": "matmul::B"
            },
            "name": "matmul",
            "outputs": {
                "C": "matmul::C"
            },
            "padding_value": 0.0,
            "tag": "MATMUL"
        },
        {
            "axis": null,
            "compute_data_type": "FLOAT",
            "elu_alpha": null,
            "inputs": {
                "IN_0": "matmul::C",
                "IN_1": "bias::IN_1"
            },
            "mode": "ADD",
            "name": "bias",
            "outputs": {
                "OUT_0": "bias::OUT_0"
            },
            